___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___

# Latent Dirichlet Allocation

## Data

We will be using articles from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv('Gospel of John - ESV.csv')

In [3]:
npr.head()

,Book,Chapter,Verse,Content
0,John,1,1,"In the beginning was the Word, and the Word wa..."
1,John,1,2,He was in the beginning with God.
2,John,1,3,"All things were made through him, and without ..."
3,John,1,4,"In him was life, and the life was the light of..."
4,John,1,5,"The light shines in the darkness, and the dark..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [5]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [6]:
dtm = cv.fit_transform(npr['Content'])

In [7]:
dtm

<879x610 sparse matrix of type '<class 'numpy.int64'>'
	with 5264 stored elements in Compressed Sparse Row format>

## LDA

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

In [9]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [10]:
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Showing Stored Words

In [11]:
len(cv.get_feature_names())

610

In [12]:
import random

In [13]:
#Dempnstation of random selection of words
for i in range(9):
    random_word_id = random.randint(0,8)
    print(cv.get_feature_names()[random_word_id])

accord
abide
account
abroad
accomplish
abide
able
abroad
abide


### Showing Top Words Per Topic

In [14]:
len(LDA.components_)

7

In [15]:
LDA.components_

array([[0.14329893, 0.14285742, 0.14285756, ..., 0.1428575 , 0.14304246,
        0.14327404],
       [9.1446448 , 3.14173016, 0.14525927, ..., 0.14285749, 0.14285726,
        0.1428574 ],
       [0.14285733, 0.14285739, 1.14127115, ..., 1.78419273, 2.14282039,
        1.14252504],
       ...,
       [0.14290482, 0.14285742, 0.14285755, ..., 0.1428575 , 0.14285727,
        0.14285741],
       [0.14285729, 0.14285735, 0.14285745, ..., 0.1431841 , 0.14285724,
        1.14277136],
       [1.14057951, 0.14285738, 1.14203954, ..., 0.14285745, 1.14270815,
        0.14285737]])

In [16]:
len(LDA.components_)


7

In [17]:
type(LDA.components_)

numpy.ndarray

In [18]:
#7 topics x 610 words
LDA.components_.shape

(7, 610)

In [19]:
single_topic = LDA.components_[0]

In [20]:
# Returns the indices of this sorted array from greatest valeus to least.
single_topic.argsort()

array([463, 205, 483,  79,  65, 302, 480, 223,  96, 150, 316, 377, 356,
        98,  33, 583, 244, 564, 118, 408, 548, 438, 290, 606, 125, 455,
       462, 348,  90, 379, 289, 385, 180, 558, 113, 262,  16, 248, 231,
        58,  23, 423, 592, 228, 102, 569, 115, 337, 255, 550, 293, 309,
       520,   7, 107, 335, 343,  20, 328, 253,  76, 221,  86, 407, 426,
       162, 366,  48,  50, 401, 219, 114, 412, 321,  74, 350, 127,  77,
       233, 334, 120, 415, 446,  69, 579, 563,  28, 171, 447, 363, 364,
       506, 134,  59, 417, 256, 330,  91, 299, 182, 165, 327,  13, 296,
       499, 345, 138, 513, 179, 161, 360, 383, 349, 170, 101, 441, 512,
        21, 589, 490, 159, 587, 516, 389, 194,  71,   5, 157, 539, 261,
       458, 116, 545, 376, 436, 177,   4, 522, 121, 585, 168, 557,  84,
       192,  34, 394, 410,  78,  95, 400, 496, 254,   9,  82,  66, 202,
        54, 523, 156, 123, 370, 404, 524, 519, 382, 492, 218, 291,  94,
       152, 418, 529, 541, 368, 593, 265, 445, 320, 531,  38,  4

In [21]:
#ARGSORT ---> returns INDEX POSITIONS SORTED FROM LEAST --> GREATEST
#Looking ror TOP 10 CALUES (10 GREATEST VALUES)
#meaning I want the LAST 10 VALUES of ARGSORT()
#it is saying GRAB the last 10 values of .argsort() (highest prob words for this topic) 
# Word most representative of this topic
single_topic[-9]

0.14287540729651876

In [22]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([ 62, 172, 443, 264, 596, 498, 213, 147, 561, 452], dtype=int64)

In [23]:
top_word_indices = single_topic.argsort()[-10:]

In [24]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

born
father
said
jesus
word
son
god
does
truly
say


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [25]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 12 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 12 WORDS FOR TOPIC #0
['sent', 'know', 'truth', 'answered', 'ask', 'born', 'father', 'said', 'jesus', 'word', 'son', 'god', 'does', 'truly', 'say']


THE TOP 12 WORDS FOR TOPIC #1
['abide', 'greater', 'sea', 'come', 'believe', 'going', 'said', 'works', 'bear', 'witness', 'light', 'answered', 'jesus', 'world', 'father']


THE TOP 12 WORDS FOR TOPIC #2
['love', 'feet', 'heard', 'came', 'lazarus', 'know', 'called', 'simon', 'jews', 'lord', 'peter', 'went', 'disciples', 'said', 'jesus']


THE TOP 12 WORDS FOR TOPIC #3
['judge', 'son', 'day', 'bread', 'eternal', 'given', 'pilate', 'god', 'jews', 'comes', 'man', 'father', 'came', 'said', 'life']


THE TOP 12 WORDS FOR TOPIC #4
['standing', 'follow', 'heard', 'told', 'man', 'answered', 'know', 'place', 'pharisees', 'eyes', 'disciples', 'things', 'believe', 'jesus', 'said']


THE TOP 12 WORDS FOR TOPIC #5
['going', 'spirit', 'speak', 'things', 'seen', 'sent', 'god', 'away', 'did', 'come', 'world', 'father', 'jesus', 'know', 'said']


T

### Attaching Discovered Topic Labels to Original Articles

In [26]:
dtm

<879x610 sparse matrix of type '<class 'numpy.int64'>'
	with 5264 stored elements in Compressed Sparse Row format>

In [27]:
dtm.shape

(879, 610)

In [28]:
len(npr)

879

In [29]:
topic_results = LDA.transform(dtm)

In [30]:
topic_results.shape

(879, 7)

In [31]:
topic_results[0]

array([0.87749543, 0.02041956, 0.02040817, 0.02042463, 0.02040817,
       0.02042142, 0.02042263])

In [32]:
topic_results[0].round(2)

array([0.88, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02])

In [33]:
topic_results[0].argmax()

0

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [34]:
npr.head()

,Book,Chapter,Verse,Content
0,John,1,1,"In the beginning was the Word, and the Word wa..."
1,John,1,2,He was in the beginning with God.
2,John,1,3,"All things were made through him, and without ..."
3,John,1,4,"In him was life, and the life was the light of..."
4,John,1,5,"The light shines in the darkness, and the dark..."


In [35]:
topic_results.argmax(axis=1)

array([0, 0, 4, 3, 1, 3, 1, 1, 1, 5, 3, 5, 3, 0, 3, 0, 6, 5, 3, 6, 6, 0,
       6, 4, 6, 3, 3, 4, 6, 3, 2, 0, 5, 0, 4, 6, 2, 5, 3, 2, 6, 2, 4, 6,
       6, 6, 6, 4, 2, 4, 0, 3, 2, 0, 6, 0, 2, 2, 5, 0, 1, 6, 2, 2, 2, 2,
       5, 4, 6, 2, 2, 2, 2, 6, 6, 6, 6, 5, 0, 2, 0, 0, 0, 5, 6, 1, 5, 4,
       3, 6, 3, 3, 5, 6, 1, 1, 1, 2, 3, 3, 2, 0, 3, 1, 6, 0, 3, 1, 6, 3,
       3, 3, 2, 5, 4, 3, 3, 5, 3, 5, 3, 6, 5, 1, 5, 3, 5, 6, 6, 6, 6, 2,
       6, 2, 3, 5, 6, 5, 6, 3, 6, 3, 2, 4, 4, 5, 4, 0, 6, 5, 6, 3, 0, 5,
       2, 0, 6, 3, 6, 4, 5, 6, 0, 3, 6, 6, 6, 2, 2, 3, 6, 1, 1, 6, 6, 6,
       6, 4, 6, 6, 6, 1, 3, 0, 1, 3, 3, 0, 3, 0, 3, 3, 0, 2, 3, 1, 1, 0,
       3, 1, 1, 0, 0, 3, 3, 3, 5, 3, 3, 0, 6, 4, 2, 1, 5, 2, 2, 4, 6, 1,
       2, 5, 4, 5, 4, 3, 6, 2, 2, 1, 1, 1, 3, 1, 2, 4, 2, 5, 0, 3, 1, 1,
       4, 2, 3, 3, 5, 3, 4, 3, 3, 3, 3, 3, 0, 1, 3, 3, 5, 0, 3, 2, 3, 3,
       3, 3, 3, 3, 3, 5, 5, 4, 4, 2, 6, 3, 4, 3, 2, 5, 3, 5, 6, 4, 2, 2,
       2, 5, 6, 6, 1, 6, 6, 6, 6, 6, 6, 6, 6, 4, 5,

In [36]:
npr['Topic'] = topic_results.argmax(axis=1)

In [37]:
npr.head(15)

,Book,Chapter,Verse,Content,Topic
0,John,1,1,"In the beginning was the Word, and the Word wa...",0
1,John,1,2,He was in the beginning with God.,0
2,John,1,3,"All things were made through him, and without ...",4
3,John,1,4,"In him was life, and the life was the light of...",3
4,John,1,5,"The light shines in the darkness, and the dark...",1
5,John,1,6,"There was a man sent from God, whose name was ...",3
6,John,1,7,"He came as a witness, to bear witness about th...",1
7,John,1,8,"He was not the light, but came to bear witness...",1
8,John,1,9,"The true light, which gives light to everyone,...",1
9,John,1,10,"He was in the world, and the world was made th...",5


## Great work!

In [45]:
import watermark
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p pandas,numpy,sklearn,watermark 

#%watermark -d -n -t -i -z -u -v -h -m -g -w


# date
print (" ")
%watermark -u -n -t -z

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

pandas   : 1.0.5
numpy    : 1.18.5
sklearn  : 0.23.1
watermark: 2.1.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 61 Stepping 4, GenuineIntel
CPU cores   : 4
Architecture: 64bit

 
Last updated: Fri Dec 04 2020 12:32:49Central Standard Time

